In [1]:
!git clone -b leo-1 --single-branch https://github.com/leopomme/NLP-Sentiment-Analysis

Cloning into 'NLP-Sentiment-Analysis'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 26 (delta 4), reused 19 (delta 3), pack-reused 0
Receiving objects: 100% (26/26), 83.37 KiB | 2.98 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
from typing import List

import torch
import itertools
from typing import Dict, List, Optional
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
from sklearn.metrics import accuracy_score
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')  # other pre-trained models like 'roberta-base'

In [3]:
import os
os.chdir('/kaggle/working/NLP-Sentiment-Analysis/src/')


In [5]:
import time, sys
import numpy as np
import argparse

import torch

from classifier import Classifier


def set_reproducible():
    # The below is necessary to have reproducible behavior.
    import random as rn
    import os
    os.environ['PYTHONHASHSEED'] = '0'
    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.
    np.random.seed(17)
    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.
    rn.seed(12345)



def load_label_output(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        return [line.strip().split("\t")[0] for line in f if line.strip()]



def eval_list(glabels, slabels):
    if (len(glabels) != len(slabels)):
        print("\nWARNING: label count in system output (%d) is different from gold label count (%d)\n" % (
        len(slabels), len(glabels)))
    n = min(len(slabels), len(glabels))
    incorrect_count = 0
    for i in range(n):
        if slabels[i] != glabels[i]: incorrect_count += 1
    acc = (n - incorrect_count) / n
    return acc*100



def train_and_eval(classifier, trainfile, devfile, testfile, run_id, device):
    print(f"\nRUN: {run_id}")
    print("  %s.1. Training the classifier..." % str(run_id))
    classifier.train(trainfile, devfile, device)
    print()
    print("  %s.2. Eval on the dev set..." % str(run_id), end="")
    slabels = classifier.predict(devfile, device)
    glabels = load_label_output(devfile)
    devacc = eval_list(glabels, slabels)
    print(" Acc.: %.2f" % devacc)
    testacc = -1
    if testfile is not None:
        # Evaluation on the test data
        print("  %s.3. Eval on the test set..." % str(run_id), end="")
        slabels = classifier.predict(testfile)
        glabels = load_label_output(testfile)
        testacc = eval_list(glabels, slabels)
        print(" Acc.: %.2f" % testacc)
    print()
    return (devacc, testacc)


if __name__ == "__main__":
    device_name = "cuda"
    device = torch.device(device_name)
    n_runs = 10
    set_reproducible()
    datadir = "../data/"
    trainfile =  datadir + "traindata.csv"
    devfile =  datadir + "devdata.csv"
    testfile = None
    # testfile = datadir + "testdata.csv"

    # Runs
    start_time = time.perf_counter()
    devaccs = []
    testaccs = []
    for i in range(1, n_runs+1):
        classifier =  Classifier()
        devacc, testacc = train_and_eval(classifier, trainfile, devfile, testfile, i, device)
        devaccs.append(np.round(devacc,2))
        testaccs.append(np.round(testacc,2))
    print('\nCompleted %d runs.' % n_runs)
    total_exec_time = (time.perf_counter() - start_time)
    print("Dev accs:", devaccs)
    print("Test accs:", testaccs)
    print()
    print("Mean Dev Acc.: %.2f (%.2f)" % (np.mean(devaccs), np.std(devaccs)))
    print("Mean Test Acc.: %.2f (%.2f)" % (np.mean(testaccs), np.std(testaccs)))
    print("\nExec time: %.2f s. ( %d per run )" % (total_exec_time, total_exec_time / n_runs))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


RUN: 1
  1.1. Training the classifier...


100%|██████████| 94/94 [00:14<00:00,  6.50it/s]


Epoch: 1,Loss: 0.6997, Dev Accuracy: 0.8351


100%|██████████| 94/94 [00:14<00:00,  6.51it/s]


...

Epoch: 8,Loss: 0.1749, Dev Accuracy: 0.8245


100%|██████████| 94/94 [00:14<00:00,  6.50it/s]


Epoch: 9,Loss: 0.1738, Dev Accuracy: 0.8298


100%|██████████| 94/94 [00:14<00:00,  6.48it/s]


Epoch: 10,Loss: 0.1546, Dev Accuracy: 0.8271

  10.2. Eval on the dev set... Acc.: 82.71


Completed 10 runs.
Dev accs: [79.79, 81.65, 82.98, 83.78, 81.12, 82.18, 82.71, 81.12, 81.91, 82.71]
Test accs: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

Mean Dev Acc.: 81.99 (1.08)
Mean Test Acc.: -1.00 (0.00)

Exec time: 1589.72 s. ( 158 per run )
